<a href="https://colab.research.google.com/github/LuciaManhique/Baseline_Portugues_Xichangana/blob/main/Model/WORKSHOP_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "por"
target_language = "tso-cha" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "workshop2" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

!mkdir -p "/content/drive/MyDrive/workshop2/$src-$tgt-$tag"
os.environ ["gdrive_path"]= "/content/drive/MyDrive/workshop2/%s-%s-%s" % (source_language, target_language, tag)

In [ ]:
!echo $gdrive_path

/content/drive/MyDrive/workshop2/por-tso-cha-workshop2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_file= "/content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Portugues.txt"
target_file= "/content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Xichangana.txt"

!wc -l $source_file
!wc -l $target_file

10145 /content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Portugues.txt
10145 /content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Xichangana.txt


In [ ]:
#pre processamento
! pip install sacremoses
tok_source_file=source_file+".tok"
tok_target_file=target_file+".tok"

#tokenizar origem
! sacremoses -l $source_language tokenize < $source_file > $tok_source_file
#tokenizar destino
! sacremoses -l $target_language tokenize < $target_file > $tok_target_file

! head $source_file*
! head $target_file*


source_file=tok_source_file
target_file=tok_target_file






     |████████████████████████████████| 895 kB 11.8 MB/s 
==> /content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Portugues.txt <==
TÍTULO I
Princípios fundamentais
CAPÍTULO I
República
ARTIGO 1.
República de Moçambique
A República de Moçambique é um Estado independente, soberano, democrático e de justiça social.
Soberania e legalidade
A soberania reside no povo.
O povo moçambicano exerce a soberania segundo as formas fixadas na Constituição.

==> /content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Portugues.txt.tok <==
TÍTULO I
Princípios fundamentais
CAPÍTULO I
República
ARTIGO 1 .
República de Moçambique
A República de Moçambique é um Estado independente , soberano , democrático e de justiça social .
Soberania e legalidade
A soberania reside no povo .
O povo moçambicano exerce a soberania segundo as formas fixadas na Constituição .
==> /content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Xichangana.txt <==
Nhlokomhaka Ya I
Sviyimiso Sva Masungulu
Ndzima Ya I
Republika
Ndzima

In [ ]:
import pandas as pd
source_file="/content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Portugues.txt.tok"
target_file="/content/drive/MyDrive/workshop2/por-tso-cha-workshop2/Xichangana.txt.tok"
max=10542
source = []
target = []
source_test = []
target_test = []
skip_lines= []

with open (source_file) as f:
  for i, line in enumerate(f):

    #skip senteces that are conteined in the test set
    if i==max:
      break

      #aplicar 20% para teste
    if i % 5==0:
      source_test.append(line.strip())
      skip_lines.append(i)
    else:
      source.append(line.strip())

with open (target_file) as f:
  for j, line in enumerate(f):  
    if j==max:
      break
         
    if j not in skip_lines:
        target.append(line.strip())
    else:
        target_test.append(line.strip()) 

print('Loaded data and skipped {}/{} lines since contained in test set'.format(len(skip_lines), i))

df = pd.DataFrame(zip(source, target), columns=['source_sentence','target_sentence'])
test = pd.DataFrame(zip(source_test, target_test), columns=['source_sentence','target_sentence'])
print("train...")
print(df.head(3))
print("test...")
print(test.head(3))








Loaded data and skipped 2030/10145 lines since contained in test set
train...
           source_sentence          target_sentence
0  Princípios fundamentais  Sviyimiso Sva Masungulu
1               CAPÍTULO I              Ndzima Ya I
2                República                Republika
test...
                                     source_sentence  \
0                                           TÍTULO I   
1                            República de Moçambique   
2  O Estado subordina-se à Constituição e funda-s...   

                                     target_sentence  
0                                   Nhlokomhaka Ya I  
1                             Republika ra Musambiki  
2  Mfumu wu le hansi ka Vumbiwo nakona masungulu ...  


In [ ]:
#pre processamento e exportar
#drop conflit translations
df_pp = df.drop_duplicates()

#apaga traducoes duplicadas
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence' , inplace=True)

df_pp =df_pp.sample(frac=1, random_state=seed ).reset_index(drop=True)




/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
import csv

num_dev_patterns = 500

if lc:
   df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
   df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

   test['source_sentence'] = test['source_sentence'].str.lower()
   test['target_sentence'] = test['target_sentence'].str.lower()

   # Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) 
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

! head train.*
! head dev.*
! head test.*        

==> train.por <==
a ) orientar a política externa ;
Temos a obrigação de agradecer sempre a Deus por vocês , irmãos . Isto é apropriado , porque a vossa fé está a crescer extraordinariamente , e o amor que todos têm uns pelos outros está a aumentar . +
Irmãos , tornem-se unidamente meus imitadores , + e continuem a observar os que andam de acordo com o exemplo que nós vos demos .
E esqueceram-se totalmente da exortação que vos é dirigida como a filhos : “ Meu filho , não menosprezes a disciplina da parte de Jeová * nem desanimes quando és corrigido por ele .
No entanto , se foram levantados com o Cristo , + continuem a buscar as coisas de cima , onde o Cristo está sentado à direita de Deus . +
4 . Se as matérias referidas no n.º 2 do artigo 179 forem objecto de convenção internacional podem ser submetidas a referendo , salvo se forem relativas à paz e à rectificação de fronteiras .
“ Vão , apresentem-se no templo e continuem a transmitir ao povo toda a mensagem a respeito da vida . ”
D

In [ ]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3312, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 3312 (delta 210), reused 257 (delta 153), pack-reused 2951
Receiving objects: 100% (3312/3312), 8.22 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (2239/2239), done.
Processing /content/joeynmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 90 kB 6.3 MB/s 
     |████████████████████████████████| 596 kB 26.6 MB/s 
     |████████████████████████████████| 436 kB 45.8 MB/s 
     |████████████████████████████████| 250 kB 51.6 MB/s

In [ ]:
from os import path
os.environ["src"] = source_language
os.environ["tgt"] = target_language

os.environ["data_path"] = path.join("joeynmt", "data", source_language+target_language)
! cat vocab.$src
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

! mkdir -p $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

! cp train.* $gdrive_path
! cp test.* $gdrive_path
! cp dev.* $gdrive_path
! cp bpe.codes.4000 $gdrive_path
! ls $gdrive_path

! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

! echo "BPE Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt

cat: vocab.por: No such file or directory
100% 4000/4000 [00:05<00:00, 754.34it/s] 
bpe.codes.4000	dev.bpe.tso-cha  dev.tso-cha   test.bpe.tso-cha  test.tso-cha
dev.bpe.por	dev.por		 test.bpe.por  test.por
bpe.codes.4000	 Portugues.txt	    test.tso-cha       Xichangana.txt
dev.bpe.por	 Portugues.txt.tok  train.bpe.por      Xichangana.txt.tok
dev.bpe.tso-cha  test.bpe.por	    train.bpe.tso-cha
dev.por		 test.bpe.tso-cha   train.por
dev.tso-cha	 test.por	    train.tso-cha
Traceback (most recent call last):
  File "joeynmt/scripts/build_vocab.py", line 62, in <module>
    build_vocab(args.train_paths, args.output_path)
  File "joeynmt/scripts/build_vocab.py", line 22, in build_vocab
    with open(path, encoding="utf-8", mode="r") as f:
FileNotFoundError: [Errno 2] No such file or directory: 'joeynmt/data/portso-cha/train.bpe.por'
BPE Sentences
Se ntsumi yi ni komba n@@ ambu wa mati ya wutomi , wa ku basa wu tlhela wu phatima , * na wu huma hi ka xitshamu xa wuhosi xa Xikwembu ni xa Xinyem

In [ ]:
from os import path
os.environ["src"] = source_language
os.environ["tgt"] = target_language
 
os.environ["data_path"] = path.join("joeynmt", "data", source_language+target_language)
! cat vocab.$src
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt
 
 
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

! mkdir -p $data_path
! cp train.* $data_path
! cp dev.* $data_path
! cp test.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

! cp train.* $gdrive_path 
! cp dev.* $gdrive_path
! cp test.* $gdrive_path
! cp bpe.codes.4000 $gdrive_path
! ls $gdrive_path

! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

! echo "BPE Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt

, 14073
. 6837
+ 5920
e 5888
a 5710
que 5312
de 4974
o 4385
os 2365
do 2032
da 1951
: 1947
em 1927
para 1827
as 1774
não 1744
* 1663
“ 1559
é 1446
” 1390
com 1359
ele 1157
se 1140
por 1090
um 1063
Deus 1024
; 947
ar 942
dos 937
no 930
ou 890
mas 820
como 804
? 744
Jesus 707
na 689
uma 624
eu 618
ado 608
à 605
ao 601
am 599
in@@ 580
d@@ 580
das 569
c@@ 543
ados 537
E 532
O 528
t@@ 513
re@@ 513
vocês 505
coisas 501
) 499
es 492
i@@ 490
sua 454
p@@ 453
en@@ 447
são 439
a@@ 435
também 435
o@@ 415
A@@ 414
eles 409
seu 404
aos 399
homem 396
ação 395
s@@ 391
al@@ 391
Cristo 385
Pois 382
aram 377
g@@ 374
’ 372
porque 369
todos 369
‘ 368
A 368
am@@ 362
to 361
disse 353
di@@ 353
er@@ 348
nos 347
vos 346
ser 344
e@@ 344
vi@@ 344
ta 343
está 342
ão 340
v@@ 337
f@@ 336
com@@ 335
quem 333
foi 333
Mas 330
ar@@ 329
r@@ 328
quando 324
Senhor 324
ia 323
í@@ 319
al 318
Então 318
! 313
pois 307
me 303
Ele 299
meio 295
de@@ 293
sobre 291
te 291
qu@@ 289
S@@ 288
Não 282
seus 282
an@@ 281
per@@ 278
ção 275
(

In [ ]:
name = '%s%s' % ( source_language,target_language)

gdrive_path=os.environ["gdrive_path"]

config = """
name: "{name}_transformer"

data:
  src: "{source_language}"
  trg: "{target_language}"
  train: "data/{name}/train.bpe"
  dev:   "data/{name}/dev.bpe"
  test:  "data/{name}/test.bpe"
  level: "bpe"
  lowercase: False
  max_sent_length: 100
  src_vocab: "data/{name}/vocab.txt"
  trg_vocab: "data/{name}/vocab.txt"

testing:
  beam_size: 5
  alpha: 1.0

training:
  #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
  random_seed: 42
  optimizer: "adam"
  normalization: "tokens"
  adam_betas: [0.9, 0.999] 
  scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
  patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
  learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
  learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
  decrease_factor: 0.7
  loss: "crossentropy"
  learning_rate: 0.0003
  learning_rate_min: 0.00000001
  weight_decay: 0.0
  label_smoothing: 0.1
  batch_size: 4096
  batch_type: "token"
  eval_batch_size: 3600
  eval_batch_type: "token"
  batch_multiplier: 1
  early_stopping_metric: "ppl"
  epochs: 100               # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
  validation_freq: 100         # TODO: Set to at least once per epoch.
  logging_freq: 100
  eval_metric: "bleu"
  model_dir: "models/{name}_transformer"
  overwrite: True              # TODO: Set to True if you want to overwrite possibly existing models. 
  shuffle: True
  use_cuda: True
  max_output_length: 100
  print_valid_sents: [0, 1, 2, 3]
  keep_last_ckpts: 3

model:
  initializer: "xavier"
  bias_initializer: "zeros"
  init_gain: 1.0
  embed_initializer: "xavier"
  embed_init_gain: 1.0
  tied_embeddings: True
  tied_softmax: True
  encoder:
      type: "transformer"
      num_layers: 6
      num_heads: 4             # TODO: Increase to 8 for larger data.
      embeddings:
          embedding_dim: 256   # TODO: Increase to 512 for larger data.
          scale: True
          dropout: 0.2
      # typically ff_size = 4 x hidden_size
      hidden_size: 256         # TODO: Increase to 512 for larger data.
      ff_size: 1024            # TODO: Increase to 2048 for larger data.
      dropout: 0.2
  decoder:
      type: "transformer"
      num_layers: 6
      num_heads: 4              # TODO: Increase to 8 for larger data.
      embeddings:
          embedding_dim: 256    # TODO: Increase to 512 for larger data.
          scale: True
          dropout: 0.2
      # typically ff_size = 4 x hidden_size
      hidden_size: 256         # TODO: Increase to 512 for larger data.
      ff_size: 1024            # TODO: Increase to 2048 for larger data.
      dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)



In [ ]:
!echo $src
!echo $tgt

por
tso-cha


In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2022-03-26 09:39:16,065 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-03-26 09:39:16,128 - INFO - joeynmt.data - Loading training data...
2022-03-26 09:39:16,290 - INFO - joeynmt.data - Building vocabulary...
2022-03-26 09:39:16,558 - INFO - joeynmt.data - Loading dev data...
2022-03-26 09:39:16,565 - INFO - joeynmt.data - Loading test data...
2022-03-26 09:39:16,592 - INFO - joeynmt.data - Data loaded.
2022-03-26 09:39:16,592 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-03-26 09:39:16,994 - INFO - joeynmt.model - Enc-dec model built.
2022-03-26 09:39:19,454 - INFO - joeynmt.training - Total params: 12104704
2022-03-26 09:39:19,457 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2022-03-26 09:39:30,009 - INFO - joeynmt.helpers -                           cfg.name : portso-cha_transformer
2022-03-26 09:39:30,009 - INFO - joeynmt.helpers -                       cfg.data.src : por
2022-03-26 

In [ ]:
! mkdir -p $gdrive_path/models/${src}${tgt}_transformer

In [ ]:
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [ ]:
! cat "$gdrive_path/models/${src${tgt}_transformer/validations.txt"

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2022-03-26 12:33:35,209 - INFO - root - Hello! This is Joey-NMT (version 1.5.1).
2022-03-26 12:33:35,209 - INFO - joeynmt.data - Building vocabulary...
2022-03-26 12:33:35,607 - INFO - joeynmt.data - Loading dev data...
2022-03-26 12:33:35,627 - INFO - joeynmt.data - Loading test data...
2022-03-26 12:33:35,681 - INFO - joeynmt.data - Data loaded.
2022-03-26 12:33:35,717 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 3600
2022-03-26 12:33:35,717 - INFO - joeynmt.prediction - Loading model from models/portso-cha_transformer/latest.ckpt
2022-03-26 12:33:39,308 - INFO - joeynmt.model - Building an encoder-decoder model...
2022-03-26 12:33:39,694 - INFO - joeynmt.model - Enc-dec model built.
2022-03-26 12:33:39,764 - INFO - joeynmt.prediction - Decoding on dev set (data/portso-cha/dev.bpe.tso-cha)...
2022-03-26 12:34:27,711 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2022-03-26 12:34:27,711 - WARNING - sacrebleu - It l

In [ ]:
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"